In [ ]:
import os,sys,inspect
# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# parentdir = os.path.dirname(parentdir)
parentdir = "/Users/paoloandreini/Desktop/github_repo/DDFM_correction_paper/DDFM"
sys.path.insert(0,parentdir)
import warnings
warnings.filterwarnings("ignore")

In [ ]:
os.getcwd()

In [ ]:
import pandas as pd
import numpy as np
from examples.empirical.implementation.bai_ng_models import BaiNgModels
import tqdm

from pandas.tseries.offsets import MonthEnd
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
start_date=pd.Timestamp("1990-01-01")

In [ ]:
data_m_all = pd.read_csv(os.path.join(parentdir, "examples", "empirical","data","mdfred_snapshot_monthly.csv"))
transform_code_m = data_m_all.iloc[0,1:]
transform_code_m = transform_code_m.astype(int)
data_m = data_m_all.iloc[1:,:].set_index("sasdate")
data_m.index = pd.to_datetime(data_m.index)
# convert to month end
data_m.index = data_m.index + MonthEnd()
data_m = data_m[data_m.index>=start_date]

In [ ]:
# pca_type:
# classic PCA: PC and PC2
# LarsPCA: LA - 5 variables, LAPC - 30 variables from X, LASPC 30 variables form X and X**2
# targeted predictors: TPC- targeted preditors from X, and TPC2 - target targeted preditors from X adn X**2
model_names = ["PC", "TPC", "TSPC", "TPC2", "PC2", "LAPC"]
forecast={"t1":1, "t6":6, "t12": 12, "t24": 24}

In [ ]:
# re-estimate once a yaer
months_oos = 12*19
test_size = 12
n_splits = int(months_oos/test_size)

In [ ]:
# loop on number of factors??
publication_lag = 0 # should we keep it? It could avoid fowrdard looking bias
y_all_targets = {}
rmsfe_all = {}
for target_i  in tqdm.tqdm(data_m.columns):
    print(target_i)
    y_pred_all_models = {}
    rmsfe_models = {}
    for fcst_name, fcst_lag in forecast.items():
        y_pred_all_tmp = []
        for name_i in ["TPC"]:#model_names:
            # print(target_i, name_i, fcst_lag)
            bai_ng_mdl = BaiNgModels(data_m, transform_code_m,
                        staz=True, standardize=True, target_name=target_i,
                        start_date=pd.Timestamp("1990-01-01"),
                        # models
                        n_factors=5, # maybe try 5 and 10 as they do in the paper / or we shold use the same value we use
                        model_name = name_i, lars_coef_number=None, target_freq="M",
                        thresh_tstat= 1.96, # we should use 1.25, 1.65, 2.56 in the paper
                        n_targeted_predictors=None, lags_y = 5, lags_f = 7, # 7 is actually 6):
                        )
            
            X_all = bai_ng_mdl.X_all.shift(fcst_lag+publication_lag).dropna(how="all", axis=0).fillna(0) 
            y_all = bai_ng_mdl.y_all.loc[X_all.index].fillna(0) # bai_ng_mdl.y_all.shift(-fcst_lag).dropna() we can shift back the y or forward the X, same thing.
            
            target_mean = bai_ng_mdl.mean.loc[y_all.index]
            target_std = bai_ng_mdl.sigma.loc[y_all.index]
            target_name = bai_ng_mdl.target_name
            tscv = TimeSeriesSplit(max_train_size=None, n_splits=n_splits, test_size=test_size)
            tscv_index = tscv.split(X_all)

            y_pred_all = []
            for i, (train_index, test_index) in enumerate(tscv_index):
                X = X_all.iloc[train_index]
                y = y_all.iloc[train_index]

                # fit model
                bai_ng_mdl.fit(X, y)

                # predict
                # need to create X_reg
                test_index_factors = list(train_index) + list(test_index)
                X_pred = X_all.iloc[test_index_factors]
                y_pred = y_all.iloc[test_index_factors]
                y_pred = bai_ng_mdl.predict(X_pred, y_pred)
                test_index_date = y_all.iloc[test_index].index
                # print(test_index_date)
                y_pred_all.append(y_pred.loc[test_index_date].map(np.real))
            y_pred_all = pd.DataFrame(pd.concat(y_pred_all), columns = [name_i])
            y_pred_all_tmp.append(y_pred_all)

        y_hat_all_models = pd.concat(y_pred_all_tmp, axis=1)
            
        # compute RMSE
        start_date = y_hat_all_models.index.min()
        end_date = y_hat_all_models.index.max()

        rmsfe = np.sqrt((((y_hat_all_models.T - y_all.loc[start_date:end_date]).T)**2).mean())
        rmsfe_models[fcst_name]=rmsfe
        
        y_pred_all_models[fcst_name] = y_hat_all_models.dropna(how="all", axis=0)
        y_pred_all_models_df = pd.concat(y_pred_all_models)
        y_pred_all_models_df.index.names = ["step-ahead", "timestamp"]
    
    rmsfe_all[target_i] = pd.concat(rmsfe_models)
    y_all_targets[target_i] = y_pred_all_models_df


In [ ]:
pd.concat(rmsfe_all).to_csv(f"{parentdir}/examples/empirical/results/rmsfe_publ_lag_0_all.csv")\
pd.concat(y_all_targets).to_csv(f"{parentdir}/examples/empirical/results/bai_ng_results_final_publ_lag0.csv")

In [ ]:
# loop on number of factors??
# AR
import statsmodels.api as sm
publication_lag = 0 # should we keep it? It could avoid fowrdard looking bias
y_all_targets_ar = {}
rmsfe_all_ar = {}
for target_i  in tqdm.tqdm(data_m.columns):
    print(target_i)
    y_pred_all_models = {}
    rmsfe_models = {}
    for fcst_name, fcst_lag in forecast.items():
        y_pred_all_tmp = []
        # print(target_i, name_i, fcst_lag)
        bai_ng_mdl = BaiNgModels(data_m, transform_code_m,
                    staz=True, standardize=True, target_name=target_i,
                    start_date=pd.Timestamp("1990-01-01"),
                    # models
                    n_factors=5, # maybe try 5 and 10 as they do in the paper / or we shold use the same value we use
                    model_name = "PC",
                    )
            
        X_all = bai_ng_mdl.y_all.shift(fcst_lag+publication_lag).dropna(how="all", axis=0).fillna(0) 
        y_all = bai_ng_mdl.y_all.loc[X_all.index].fillna(0) # bai_ng_mdl.y_all.shift(-fcst_lag).dropna() we can shift back the y or forward the X, same thing.

        target_name = bai_ng_mdl.target_name
        tscv = TimeSeriesSplit(max_train_size=None, n_splits=n_splits, test_size=test_size)
        tscv_index = tscv.split(X_all)

        y_pred_all = []
        for i, (train_index, test_index) in enumerate(tscv_index):
            X = X_all.iloc[train_index]
            y = y_all.iloc[train_index]
            X_pred = X_all.iloc[test_index]

            # fit model
            model = sm.OLS(y, X).fit()

            # predict
            # need to create X_reg
            y_pred = model.predict(X_pred)
            # print(test_index_date)
            y_pred_all.append(y_pred.map(np.real))
        y_pred_all = pd.DataFrame(pd.concat(y_pred_all), columns = [target_i])
        y_pred_all_tmp.append(y_pred_all)

        y_hat_all_models = pd.concat(y_pred_all_tmp, axis=1)
            
        # compute RMSE
        start_date = y_hat_all_models.index.min()
        end_date = y_hat_all_models.index.max()

        rmsfe = np.sqrt((((y_hat_all_models.T - y_all.loc[start_date:end_date]).T)**2).mean())
        rmsfe_models[fcst_name]=rmsfe
        
        y_pred_all_models[fcst_name] = y_hat_all_models.dropna(how="all", axis=0)
        
    y_pred_all_models_df = pd.concat(y_pred_all_models)
    y_pred_all_models_df.index.names = ["step-ahead", "timestamp"]
    rmsfe_all_ar[target_i] = pd.concat(rmsfe_models)
    y_all_targets_ar[target_i] = y_pred_all_models_df.squeeze()
    



In [ ]:
pd.concat(rmsfe_all_ar).to_csv(f"{parentdir}/examples/empirical/results/rmsfe_ar1_all.csv")
pd.DataFrame(y_all_targets_ar).to_csv(f"{parentdir}/examples/empirical/results/ar1_pred_all.csv")